In [1]:
from pathlib import Path
import numpy as np
import json
import cvxpy as cp

import torch
import torch
from src.problems.utils import sample_data_for_group
from src.problems.problems import compute_producer_optimal_solution, _compute_consumer_optimal_solution_cvar
from src.problems.gradient_problem import compute_consumer_optimal_solution_cvar_grad

In [2]:
DATA_PATH_ROOT = Path("../../data")

In [3]:
# load data
with open(DATA_PATH_ROOT / "amazon_predictions.npy", "rb") as f:
    REL_MATRIX = np.load(f)

with open(DATA_PATH_ROOT / "amazon_user_groups.json", "r") as f:
    GROUPS_MAP = json.load(f)

In [4]:
N_CONSUMERS = 400
N_PRODUCERS = 400

In [5]:
sampled_matrix, consumer_ids, group_assignments = sample_data_for_group(
    n_consumers=N_CONSUMERS,
    n_producers=N_PRODUCERS,
    groups_map=GROUPS_MAP,
    group_key="top_category",
    data=REL_MATRIX,
    seed=1
)

In [7]:
compute_producer_optimal_solution(
    rel_matrix=sampled_matrix,
    k_rec=10,
)

(10.0,
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], shape=(400, 400)))

In [33]:
_, cvar_allocations = _compute_consumer_optimal_solution_cvar(
    rel_matrix=sampled_matrix,
    k_rec=10,
    producer_max_min_utility=10,
    group_assignments=group_assignments,
    gamma=0.5,
    alpha=0.5,
    solver=cp.SCIP
)

In [34]:
cvar_allocations = cvar_allocations.astype(int)

In [35]:
np.mean(np.sum(cvar_allocations * sampled_matrix, axis=1)) / 10

np.float64(0.9445202392750154)

In [ ]:
allocations = compute_consumer_optimal_solution_cvar_grad(
    rel_matrix=sampled_matrix,
    k_rec=10,
    producer_max_min_utility=10,
    gamma=0.5,
    group_assignments=group_assignments,
    alpha=0.5,
    hidden_dim=200,
    max_epochs=30000,
    verbose=True,
    max_patience=5
)

Epoch     1 — loss: 36189.8164, util: 0.00, card: 36189.7539, prod: 0.0000, bin: 0.0625, rho: -0.0010, tau: 0.9990, grad: 5.0000
Epoch   500 — loss: 0.0738, util: 0.06, card: 0.0107, prod: 0.0001, bin: 0.0013, rho: -0.0060, tau: 0.6064, grad: 0.6299
Epoch  1000 — loss: 0.0575, util: 0.04, card: 0.0125, prod: 0.0000, bin: 0.0007, rho: -0.0060, tau: 0.3677, grad: 0.2614
Epoch  1500 — loss: 0.0478, util: 0.02, card: 0.0243, prod: 0.0000, bin: 0.0007, rho: -0.0060, tau: 0.2230, grad: 1.6917
Epoch  2000 — loss: 0.0300, util: 0.02, card: 0.0086, prod: 0.0000, bin: 0.0006, rho: -0.0060, tau: 0.1352, grad: 0.3926
Epoch  2500 — loss: 0.0243, util: 0.02, card: 0.0062, prod: 0.0001, bin: 0.0006, rho: -0.0060, tau: 0.0820, grad: 0.9120
Epoch  3000 — loss: 0.0233, util: 0.02, card: 0.0019, prod: 0.0005, bin: 0.0007, rho: -0.0060, tau: 0.0497, grad: 2.3523
Epoch  3500 — loss: 0.0199, util: 0.02, card: 0.0030, prod: 0.0000, bin: 0.0009, rho: -0.0060, tau: 0.0301, grad: 1.5388
Epoch  4000 — loss: 0.01

In [7]:
cvar_allocations.sum(axis=0)

NameError: name 'cvar_allocations' is not defined

In [8]:
allocations.sum(axis=0)

array([  6.301512 ,   4.9992223,   4.998426 ,   4.997872 ,   4.999021 ,
         4.998382 ,   4.99619  ,   4.9976606, 220.       ,   4.997788 ,
         5.054884 ,   4.9982195,   4.997365 ,   5.166075 ,   6.339166 ,
         4.997844 ,   4.9983544,   4.996832 ,   4.9979815,   4.9977555,
       365.67273  ,   4.9980345,   4.9999313,   5.0556755,   4.9972286,
         5.002362 ,   5.211969 ,   5.288984 ,   4.9979973,   5.024349 ,
         4.997323 ,   4.999029 ,   4.998927 ,   4.99877  ,   5.1152034,
         4.997915 ,   5.008338 ,   4.9977317,   4.998155 ,   4.998006 ,
         4.998737 ,   5.1482334,   5.1303244,   4.9976172,   4.9980426,
         4.9981513,   4.998585 ,   4.998369 ,   4.9994283,   4.997899 ,
         4.99696  ,   4.9982185,   4.998072 ,   5.0151587,   5.045986 ,
         4.996604 ,   4.998085 ,   4.99835  ,   5.1243043,   5.06973  ,
         4.998045 ,   5.1189246,   4.9983654,   4.9999375,   4.9983096,
         4.9981046,   5.1095753,   4.9976754,   4.998905 ,   4.9

In [9]:
np.round(allocations).sum(axis=0)  # check that each user has k_rec items

array([  6.,   5.,   5.,   5.,   6.,   5.,   5.,   4., 220.,   5.,   5.,
         5.,   5.,   5.,   6.,   5.,   5.,   5.,   5.,   5., 366.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5., 219.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   4.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   4.,   4.,   4.,   4.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   4.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,   5.,
         5.,   5., 220.,   5.,   5., 218.,   5.,   5.,   5.,   5.,   5.,
         5.,   5.,   5.,   5.,   5.,   5.,   5.,   

In [10]:
def compute_std_between_groups(out, group_assignments):
    unique_groups, group_indices = np.unique(group_assignments, return_inverse=True)
    num_groups = len(unique_groups)
    group_masks = [group_indices == i for i in range(num_groups)]
    group_sizes = np.array([mask.sum() for mask in group_masks])

    means = []
    for mask, size in zip(group_masks, group_sizes):
        group_alloc = out[mask]
        mean = np.mean(group_alloc.sum(axis=0))
        means.append(mean)

    return np.mean(means)

In [12]:
compute_std_between_groups(allocations * sampled_matrix, group_assignments)

np.float64(0.5155661581092854)

In [173]:
compute_std_between_groups(A_opt * sampled_matrix, group_assignments), compute_std_between_groups(cvar_allocations * sampled_matrix, group_assignments)

(np.float64(3.1376325527596727), np.float64(3.1616680929488155))

In [167]:
A_cont = model.eval().forward().detach().cpu().numpy()
A_opt = np.zeros_like(A_cont)
for i, row in enumerate(A_cont):
    A_opt[i, row.argsort()[-k_rec:]] = 1

In [168]:
cvar_allocations.sum(axis=0)

array([  5,   5,   5,   5,  94,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,  41,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,  55,   5,   5,   5,   5,
         5,   5,   5,  26,   5,   5,   5,   5,   5,   5,  15,   5,   5,
        63,   5,  16,   5,   5, 131,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   9,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,  66,   5,   5,   5,   5,
         5,   5,   5,   6,   5,  12,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5, 157,   6,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
        36,   5,   5,   5,   5,   5,   5,   5,  29,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,  1

In [170]:
A_opt.sum(axis=0).astype(int)

array([  5,   5,   5,   5,  86,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,  19,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,  74,   5,   5,   5,   5,
         5,   5,   5,   8,   5,   5,   5,   5,   4,   5,   5,   5,   5,
        12,   5,   6,   5,   5, 200,   5,   5,   5,   5,   5,   5,   4,
         5,   5,   5,   4,   5,   5,   5,   5,   4,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,  90,   5,   5,   5,   5,
         5,   5,   5,   5,   5,  12,   5,   5,   5,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
         4,   5,   5,   5, 200,   5,   5,   5,   6,   5,   5,   4,   5,
         5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
        13,   5,   5,   5,   5,   5,   5,   5,   6,   5,   5,   5,   5,
         5,   5,   5,   5,   5,   5,   4,   5,   5,   5,   5,   

In [159]:
np.mean(np.sum(A_opt * sampled_matrix, axis=1) / k_rec)

np.float64(0.9415035986031873)

In [306]:
A_opt.sum(axis=1)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10

In [317]:
compute_correlation(A_opt, mean_allocations)

np.float64(0.7347368338939393)

In [551]:

R = torch.tensor(rel_mat)
n, m = R.shape
z = torch.rand(n, m, requires_grad=True, device="cpu")
opt = torch.optim.Adam([z], lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=100, factor=0.5, verbose=True)
prod_min = 5


lambda_util = 10
lambda_card = 100
lambda_prod = 10
k_rec = 10
decay = 1e-2

for epoch in range(1, 3001):
    a = torch.sigmoid(z)

    util   = torch.mean((a * R).sum(dim=1) / k_rec)
    L_util = -util

    row_sums = a.sum(dim=1)
    L_card   = torch.mean((row_sums - k_rec)**2)

    col_sums = a.sum(dim=0)
    L_prod   = torch.mean(torch.relu(prod_min - col_sums)**2)

    L_bin  = torch.mean(a * (1 - a))

    loss = lambda_util * L_util + lambda_card * L_card + lambda_prod * L_prod + L_bin

    opt.zero_grad()
    loss.backward()

    if epoch % 500 == 0:
        print(f"Epoch {epoch:5d} — loss: {loss.item():.4f}, util: {-L_util.item():.2f}, card: {L_card.item():.4f}, prod: {L_prod.item():.4f}, bin: {L_bin.item():.4f}")

    opt.step()
    scheduler.step(loss.item())

A_cont = torch.sigmoid(z).detach().cpu().numpy()
# projection to binary
A_opt = np.zeros_like(A_cont)
for i, row in enumerate(A_cont):
    A_opt[i, row.argsort()[-k_rec:]] = 1


/Users/dominykas.seputis/github/msc-thesis/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch   500 — loss: -8.8511, util: 0.89, card: 0.0004, prod: 0.0000, bin: 0.0476
Epoch  1000 — loss: -8.8768, util: 0.89, card: 0.0000, prod: 0.0000, bin: 0.0475
Epoch  1500 — loss: -8.8769, util: 0.89, card: 0.0000, prod: 0.0000, bin: 0.0475
Epoch  2000 — loss: -8.8771, util: 0.89, card: 0.0000, prod: 0.0000, bin: 0.0475
Epoch  2500 — loss: -8.8773, util: 0.89, card: 0.0000, prod: 0.0000, bin: 0.0475
Epoch  3000 — loss: -8.8776, util: 0.89, card: 0.0000, prod: 0.0000, bin: 0.0475


In [553]:
np.sort(A_cont[0])[::-1]

array([0.05779154, 0.05771403, 0.05765368, 0.05763114, 0.05748032,
       0.05746737, 0.05745109, 0.0573871 , 0.05737695, 0.05736849,
       0.05735333, 0.0573409 , 0.05724973, 0.05698105, 0.0569311 ,
       0.05686256, 0.05677284, 0.05677127, 0.05671323, 0.0565905 ,
       0.05654738, 0.05640062, 0.0563875 , 0.05636477, 0.05631623,
       0.05626244, 0.0561929 , 0.05607791, 0.0560635 , 0.05598119,
       0.055796  , 0.05579099, 0.05566983, 0.05551579, 0.05549545,
       0.05544922, 0.05526333, 0.05515146, 0.05514026, 0.05512378,
       0.0551208 , 0.05511221, 0.05501353, 0.05500968, 0.0549847 ,
       0.0549709 , 0.05496861, 0.05494679, 0.05487379, 0.05481775,
       0.05479058, 0.05460422, 0.05456612, 0.05438355, 0.05427985,
       0.0542602 , 0.05414438, 0.05396806, 0.0538949 , 0.05344047,
       0.05329363, 0.05327445, 0.05327269, 0.05324512, 0.05314672,
       0.05307278, 0.0529814 , 0.05295432, 0.05279669, 0.05271975,
       0.05261503, 0.05256554, 0.05239647, 0.05221638, 0.05215

In [520]:
np.mean(np.sum(A_opt * rel_mat, axis=1) / k_rec)

np.float64(0.9046699930702641)

In [524]:
compute_correlation(mean_allocations, A_opt)

np.float64(0.023157894478032476)

In [522]:
A_opt.sum(axis=0)

array([ 9.,  9.,  4.,  8., 13., 13., 10.,  6.,  6., 12., 11., 11., 21.,
        4.,  7.,  8.,  6.,  8., 16.,  5., 15., 13., 20.,  9., 12., 10.,
        6.,  2., 11.,  9., 14., 17., 21., 21., 10.,  8., 11.,  5.,  9.,
       12.,  9.,  9., 14., 12., 13.,  7.,  9.,  3.,  8.,  6.,  9., 14.,
        8., 12.,  9., 15.,  7., 19., 10.,  5., 14.,  9., 10.,  9., 12.,
       15.,  9., 11., 10.,  9., 13., 10.,  6., 10.,  7., 10.,  6., 10.,
       14.,  6.,  9., 12., 13.,  4.,  9.,  7.,  6.,  9.,  9.,  8., 14.,
        9.,  9., 10.,  5., 16., 11.,  8., 10.,  7.,  9., 18., 12.,  4.,
       11., 10., 12.,  9.,  4., 18., 14., 18., 17., 16.,  8.,  8., 15.,
        7., 11.,  8., 10., 12.,  7.,  6.,  8., 12.,  3., 10.,  4.,  9.,
        9., 11., 10., 14.,  4.,  9., 10., 12., 10., 10.,  6., 13.,  9.,
       11.,  3.,  7.,  3.,  9.,  8., 14.,  6., 14.,  9.,  9., 10., 17.,
       18., 11.,  5.,  9.,  9., 10., 11., 12., 16., 14., 14.,  1.,  5.,
       13., 12.,  5., 11., 10.,  8., 12., 16., 13.,  6., 13.,  7

In [386]:
n, m = rel_mat.shape  # ensure rel_matrix is defined
R = torch.tensor(rel_mat, dtype=torch.float32, device="cpu")

z = (torch.randn(n, m, device="cpu") * 0.01 + 0.1).requires_grad_(True)
log_lambda_prod = torch.zeros(m, device="cpu", requires_grad=False)

opt = torch.optim.Adam([z], lr=0.05)


# STAGE 1: Allow the model to find a meaningful initial solution (no lambdas initially)
for epoch in range(1, 1001):
    tau = max(0.05, 0.995 ** epoch)
    a = torch.sigmoid(z / tau)

    util = torch.mean((a * R).sum(dim=1) / k_rec)
    L_util = -util

    row_sums = a.sum(dim=1)
    L_card = ((row_sums - k_rec)**2).mean()

    col_sums = a.sum(dim=0)
    prod_shortfall = torch.relu(prod_min - col_sums)
    L_prod = (prod_shortfall**2).mean()  # soft quadratic penalty, no lambda yet

    loss = L_util + 5.0 * L_card + 5.0 * L_prod  # balanced weights, moderate penalties

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 500 == 0:
        violated_count = (col_sums.detach().cpu().numpy() < prod_min).sum()
        print(f"[Stage 1] Epoch {epoch:4d} | Loss: {loss.item():.3f} | "
              f"Util: {-L_util.item():.3f} | Violations: {violated_count}")

# STAGE 2: Now introduce controlled dual updates to enforce constraints strictly
for epoch in range(1001, 4001):
    tau = max(0.01, 0.995 ** epoch)
    a = torch.sigmoid(z / tau)

    util = torch.mean((a * R).sum(dim=1) / k_rec)
    L_util = -util

    row_sums = a.sum(dim=1)
    L_card = ((row_sums - k_rec)**2).mean()

    col_sums = a.sum(dim=0)
    prod_shortfall = torch.relu(prod_min - col_sums)

    lambda_prod = torch.exp(log_lambda_prod)
    L_prod = (lambda_prod * (prod_shortfall**2)).mean()

    loss = L_util + 10.0 * L_card + L_prod  # slightly stronger constraints now

    opt.zero_grad()
    loss.backward()
    opt.step()

    with torch.no_grad():
        log_lambda_prod += 0.01 * prod_shortfall  # controlled dual updates
        log_lambda_prod.clamp_(min=-2, max=5)

    if epoch % 500 == 0:
        violated_count = (col_sums.detach().cpu().numpy() < prod_min).sum()
        avg_shortfall = prod_shortfall.mean().item()
        print(f"[Stage 2] Epoch {epoch:4d} | Loss: {loss.item():.3f} | "
              f"Util: {-L_util.item():.3f} | Violations: {violated_count} | Avg Shortfall: {avg_shortfall:.3f}")

# Final binary projection
A_cont = torch.sigmoid(z).detach().cpu().numpy()
A_opt = np.zeros_like(A_cont)
for i, row in enumerate(A_cont):
    A_opt[i, row.argsort()[-k_rec:]] = 1

producer_counts = A_opt.sum(axis=0)
violations = (producer_counts < prod_min).sum()
print(f"Final violations after projection: {violations}")


[Stage 1] Epoch  500 | Loss: 625.000 | Util: 0.000 | Violations: 200
[Stage 1] Epoch 1000 | Loss: 625.000 | Util: 0.000 | Violations: 200
[Stage 2] Epoch 1500 | Loss: 4710.328 | Util: 0.000 | Violations: 200 | Avg Shortfall: 5.000
[Stage 2] Epoch 2000 | Loss: 4710.328 | Util: 0.000 | Violations: 200 | Avg Shortfall: 5.000
[Stage 2] Epoch 2500 | Loss: 4710.328 | Util: 0.000 | Violations: 200 | Avg Shortfall: 5.000
[Stage 2] Epoch 3000 | Loss: 4710.328 | Util: 0.000 | Violations: 200 | Avg Shortfall: 5.000
[Stage 2] Epoch 3500 | Loss: 4710.328 | Util: 0.000 | Violations: 200 | Avg Shortfall: 5.000
[Stage 2] Epoch 4000 | Loss: 4710.328 | Util: 0.000 | Violations: 200 | Avg Shortfall: 5.000
Final violations after projection: 5


In [387]:
np.mean(np.sum(A_opt * rel_mat, axis=1) / k_rec)

np.float64(0.8925312649793051)

In [388]:
A_opt.sum(axis=0)

array([ 7.,  5., 12.,  9., 13.,  8., 12.,  7., 11.,  6.,  9., 16., 13.,
        9.,  8.,  4., 12.,  8.,  9., 12.,  9., 10., 11., 13., 12.,  7.,
       11., 10., 10.,  9., 12., 11.,  9., 11., 10., 11., 11., 13.,  7.,
        5.,  7.,  8., 14.,  6.,  3., 16., 11., 12.,  8., 13., 10., 13.,
        7.,  8.,  6.,  6., 15., 11., 13., 11., 13., 13.,  4.,  9., 16.,
       14., 13., 11.,  8., 12., 10., 11., 11., 10.,  9.,  7., 18., 10.,
        9., 10., 15., 13.,  8., 11.,  9., 11.,  7., 12., 16., 11., 11.,
       10., 13.,  7., 10., 14., 12.,  8., 10., 10.,  6., 14., 15.,  6.,
       15., 17.,  6., 18., 12.,  9., 13., 13., 11., 10., 10., 15.,  8.,
       10., 11.,  8., 14.,  9., 11.,  7.,  9., 13., 11., 11.,  5.,  6.,
        7.,  6.,  8.,  8.,  9.,  9., 10.,  4., 11., 13., 11.,  6.,  9.,
        6.,  6.,  9.,  7.,  9.,  7., 10.,  6., 17., 11.,  3.,  7., 19.,
        7., 10.,  9., 11., 10.,  6.,  9., 11.,  6.,  9., 11., 13.,  9.,
        8.,  8.,  9.,  5., 11., 11., 15.,  7.,  9., 12., 12., 10

In [281]:
np.mean(np.sum(mean_allocations * rel_mat, axis=1) / k_rec)

np.float64(0.9466594812127047)

In [276]:

# for each rel_mat row, take top-k based on the allocation
sorted_indices = np.argsort(mean_allocations, axis=1)[::-1][:, :k_rec]
print(np.mean(rel_mat[np.arange(rel_mat.shape[0])[:, None], sorted_indices], axis=1))
print(np.sum(mean_allocations, axis=0))
print(np.sum(mean_allocations, axis=1))


[0.88454426 0.82073556 0.92958885 0.95041268 0.90130418 0.95360957
 0.90437249 0.90215369 0.84169271 0.81099343 0.74843775 0.96578593
 0.94725356 0.95360391 0.89828264 0.8910981  0.78405916 0.89805986
 0.94508075 0.97740006 0.79811105 0.87104322 0.98142324 0.85312095
 0.8231652  0.91341474 0.89999115 0.8845513  0.9452226  0.85785696
 0.91790804 0.97983441 0.91876659 0.89139027 0.89914497 0.8987039
 0.81594811 0.89713778 0.82567525 0.91429423 0.9268033  0.91292709
 0.90771376 0.97021217 0.92682126 0.94819362 0.8867817  0.87495038
 0.93166013 0.7806246  0.73209766 0.78123458 0.92761685 0.86109238
 0.96749913 0.79566761 0.89570176 0.93509685 0.9707759  0.72826213
 0.84841753 0.72617826 0.92275587 0.95187544 0.91133795 0.87837534
 0.89899835 0.80371097 0.71420634 0.93934083 0.80872552 0.78192086
 0.92679767 0.93672771 0.96834136 0.74314832 0.73710272 0.84708724
 0.88626303 0.84994237 0.88079671 0.79758221 0.81891578 0.87057058
 0.97260963 0.97045069 0.7622217  0.96137044 0.73829369 0.96344

In [182]:
# compute correlation between the two allocations


corr = compute_correlation(new_alls, mean_allocations)
corr

np.float64(0.3194736703774324)